In [48]:
import pandas as pd
import numpy as np
import os
import math

#### Make Ventilation General Info from Chart Events

In [16]:
df_procedure = pd.read_csv(f"{input_file}icu/procedureevents.csv.gz", compression='gzip')
df_patient = pd.read_csv(output_file + "patient_info.csv")

In [17]:
input_file = 'Data/MIMIC/mimic-iv-2.2/'
output_file = input_file

df_items = pd.read_csv(f"{input_file}icu/d_items.csv")
df_items['label'] = df_items['label'].apply(str.lower)

df_items_filt = df_items[df_items['label'].str.contains('ventil')]

procedure_items = df_items_filt[df_items_filt['linksto'] == 'procedureevents']['itemid'].values
chart_items = df_items_filt[df_items_filt['linksto'] == 'chartevents']['itemid'].values

In [45]:
df_proc_vent_info = df_procedure[df_procedure['itemid'].isin(procedure_items)]
patient_stays = df_patient['stay_id'].values
df_proc_vent_info_stays = df_proc_vent_info[df_proc_vent_info['stay_id'].isin(patient_stays)].sort_values(by=['stay_id'])
df_patient = df_patient.sort_values(by=['stay_id'])
df_proc_vent_info_stays = pd.merge(df_proc_vent_info_stays, df_patient[['stay_id', 'intime', 'outtime']])

df_proc_vent_info_stays['starttime'] = pd.to_datetime(df_proc_vent_info_stays['starttime'])
df_proc_vent_info_stays['endtime'] = pd.to_datetime(df_proc_vent_info_stays['endtime'])
df_proc_vent_info_stays['intime'] = pd.to_datetime(df_proc_vent_info_stays['intime'])
df_proc_vent_info_stays['outtime'] = pd.to_datetime(df_proc_vent_info_stays['outtime'])
df_proc_vent_info_stays['start_from'] = (df_proc_vent_info_stays['starttime'] - df_proc_vent_info_stays['intime']) / np.timedelta64(24, 'h')
df_proc_vent_info_stays['time_used'] = (df_proc_vent_info_stays['endtime'] - df_proc_vent_info_stays['starttime']) / np.timedelta64(24, 'h')

In [51]:
df_proc_vent_info_stays['day_start'] = df_proc_vent_info_stays['start_from'].apply(math.ceil)
df_proc_vent_info_stays['time_used_days'] = df_proc_vent_info_stays['time_used'].apply(math.ceil)
df_vent_final = df_proc_vent_info_stays[['subject_id', 'hadm_id', 'stay_id', 'day_start', 'time_used_days']]

In [54]:
# Save file
df_vent_final.to_csv(output_file + "ventilation_info.csv", index=False)